In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv(r"C:\Users\data\maindataset.csv", encoding='utf-8')
df.head(3)

In [ ]:
# ad 테이블에 해당하는 컬럼들만 가진 데이터 프레임 생성
ad_df = df.iloc[:, 13:22]
ad_df = pd.concat([df['content_id'], ad_df], axis=1)
ad_df.info()

In [ ]:
ad_df.head(3)

In [ ]:
# 문자열 데이터를 가진 컬럼들은 제거한 imputer 전용 데이터 생성
df_impute = ad_df.drop(columns = ['name', 'keyword'])
df_impute.isnull().sum()

In [ ]:
df_impute.category_id_1.value_counts()

In [ ]:
df_impute['comment_cnt'].value_counts()

In [ ]:
# KNN Imputer를 위한 최적의 파라미터 GridSearch 이용
param_grid = [
    {'n_neighbors': range(3, 11, 2)}
]

imputer = KNNImputer(missing_values=np.nan, add_indicator=True)
gs = GridSearchCV(imputer, param_grid = param_grid, cv=10, n_jobs = -1, scoring = 'f1')#score: r2, f1
gs.fit(df_impute)

# 가장 좋은 성능을 보인 파라미터 조합의 estimator을 model에 저장
model = gs.best_estimator_

In [ ]:
model #f1 scoring일 때 최적 모델 값

In [ ]:
# f1 score일 때 결측값 채우기
df_impute = model.transform(df_impute) #np.array형의 데이터 출력
df_impute

In [ ]:
# np.array 형태의 결측값 채운 데이터를 ad테이블의 컬럼명을 가진 데이터 프레임으로 만들어주기
f1_impute = pd.DataFrame(df_impute).iloc[:, :8].rename(columns = {n: c for n, c in zip(range(1, 8), col_names)}).rename(columns = {0: ad_df.columns[0]})
f1_impute

In [ ]:
# 결측치 부분이 제대로 채워졌는지 해당 부분 확인하기
f1_impute[(f1_impute.category_id_1 == 0)]

In [ ]:
f1_impute['comment_cnt'].value_counts()